# Практическая работа по анализу текста

В качестве метрики качества используйте отчет о классификации https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.classification_report.html

In [ ]:
import os
import string
import razdel
import math
import random
import pymorphy2
import nltk

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook as tqdm
from tqdm.notebook import trange

from gensim import corpora, models, similarities, downloader

tqdm.pandas()

In [ ]:
SEED = 1234
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

## 1. Обработка данных

In [ ]:
morph = (
    pymorphy2.MorphAnalyzer()
)  # инициализируйется вне функции, иначе код будет работать дольше
# spaCy если нет возможности обработки английского языка

# Напишите функцию обработки предложения, используя следующие библиотеки pymorphy2, nltk, razdel. Обязательно ознакомьтесь с каждой из библиотек
# после написания функции примените ее к датасету, используя df.progress_apply(...)
# функция последовательно должна делать следующие шаги: разбиение текста на токены с помощью razdel, приведение их к нормальной форме с помощью morph и затем обработка стоп-слов с помощью nltk.
# Вы можете внести изменения в описанный выше пайплайн, добавив новые шаги, но уменьшать перечисленные нельзя.
# пайплайн выше приспособлен к работе с кириллицей. В датасете есть английские слова, их тоже надо суметь обработать
# (одна из библиотек умеет автоматически обнаруживать иностранные слова и возвращать соответствующий тег).

## 2. EDA

In [ ]:
# Проанализируйте классы, их распределение и количество объектов в каждом. Оцените необходимость балансировки.

In [ ]:
# Постройте облака точек https://www.datacamp.com/tutorial/wordcloud-python

In [ ]:
# поищите выбросы в токенах, возможно, где-то вместо кириллицы использовалась латиница, тогда символы x и х будут разными токенами, а смысл может быть у них одинаковый.

In [ ]:
# разбейте выборку на тест и трейн train_test_split(X, y, random_state=SEED, stratify=True)

In [ ]:
# поищите выбросы в предложениях, это можно сделать не на обработанных данных. Возможно, где-то перепутаны классы, или есть пустые предложения. Этот шаг обязательно после разбиения и только на трейне.

# 3. Тематическое моделирование

https://medium.com/@sayahfares19/text-analysis-topic-modelling-with-spacy-gensim-4cd92ef06e06

https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py

In [ ]:
# используя ссылки выше создайте тематическую модель над предложениями. Помните, что темы != классы, поэтому их количество не обязательно должно совпадать.
# не забывайте использовать фильтры над словами, это очень важно, чтобы не учиться на слишком частых токенах и не учитывать крайне редкие.
# обязательно добавляйте n-граммы в модель.
# для всего вышеперечисленного в gensim есть методы, параметры и функции

In [ ]:
# Train LDA model.
from gensim.models import LdaModel
from gensim.models import CoherenceModel


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics
    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token
    coherence_values = []
    model_list = []

    for num_topics in tqdm(range(start, limit, step)):
        model = LdaModel(
            corpus=corpus,
            id2word=id2word,
            chunksize=chunksize,
            alpha="auto",
            eta="auto",
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=eval_every,
        )
        model_list.append(model)
        coherencemodel = CoherenceModel(
            model=model, texts=texts, dictionary=dictionary, coherence="c_v"
        )
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
# Can take a long time to run.

# Set training parameters.
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.
limit=10; start=2; step=1;


model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=train, start=, limit=, step=)

In [ ]:
# Show graph
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc="best")
plt.show()

In [ ]:
# Выберите модель с наибольшей когерентностью.
# Визуализируйте темы полученной модели с помощью pyLDAvis

In [ ]:
# Используйте выход модели (вероятности принадлежности к каждой из тем), как входные фичи, например, в SVM для предсказания классов предложений.
# Токены и их векторные описания в таком случае не используются. Измерьте качество полученного алгоритма на тесте и трейне

## 4. Градиентный бустинг

In [ ]:
import catboost
import optuna

1. В методах преобразования данных пробуйте различные параметры, в поисках лучших для решения текущей задачи
2. Не забывайте про подбор параметров у самого бустинга с помощью optuna

https://forecastegy.com/posts/catboost-hyperparameter-tuning-guide-with-optuna/

### Мешок слов

In [ ]:
# обучите градиентный бустинг на мешке слов https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

### Мешок слов + n-grams

In [ ]:
# обучите градиентный бустинг на мешке слов с n-граммами https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html с n-граммами

### Мешок слов + m-skip-n-grams

In [ ]:
from toolz import itertoolz, compose
from toolz.curried import map as cmap, sliding_window, pluck
from sklearn.feature_extraction.text import CountVectorizer


class SkipGramVectorizer(CountVectorizer):
    def build_analyzer(self):
        preprocess = self.build_preprocessor()
        stop_words = self.get_stop_words()
        tokenize = self.build_tokenizer()
        return lambda doc: self._word_skip_grams(
            compose(tokenize, preprocess, self.decode)(doc), stop_words
        )

    def _word_skip_grams(self, tokens, stop_words=None):
        # handle stop words
        if stop_words is not None:
            tokens = [w for w in tokens if w not in stop_words]

        return compose(cmap(" ".join), pluck([0, 2]), sliding_window(3))(tokens)

### TF-IDF

In [ ]:
# обучите градиентный бустинг на мешке слов https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

### TF-IDF + n-grams

### TF-IDF + m-skip-n-grams

In [ ]:
# по аналогии с кодом выше

### Генерация искусственных данных и балансировка классов

Выберите лучшее представление данных, опираясь на метрику. Попробуйте сбалансировать классы с помощью весов (параметр catboost), если выше этого не делали. Попробуйте сгенерировать новые данные для классов, в которых меньше всего объектов. Генерация представляет собой семплирование токенов из всего множества токенов определенного класса. Обучите модель на новом датасете, сравните качество

1. признаки предложений (длина предложений, наличие орф.знаков, количество предложений в тексте)
2. обработка опечаток (расстояние между словом нормальным и словом с опечаткой)
3. склеивание с частицей не.

In [ ]:
#

In [ ]:
# прогнать флоу выше с catboost

## Понижение размерности

In [ ]:
# уменьшите размерность векторов с помощью PCA, посмотрите, улучшается ли качество. Попробуйте несколько вариантов понижения размерности: от 90% до 50% от изначального размера вектора.

In [ ]:
# (ПО ЖЕЛАНИЮ) применить метод кластеризации и использовать метки кластеров как доп признак

# 5. Интерпретация результатов

https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Catboost%20tutorial.html

In [ ]:
# используя туториал выше, интерпретируйте результаты. Определите, какие слова влияют в каждом классе больше остальных.

## Визуализация результатов

In [ ]:
# С помощью методов понижения размерности T-SNE И U-MAP взгляните на получившиеся векторные представления данных